# Notebook 01 — Pipeline mínima de certificação (Excel → SQLite)

**Objetivo (nível estágio):** transformar a planilha “bruta” em uma **fonte única confiável** no SQLite, com:
- leitura das abas do Excel
- padronização de nomes
- tipagem básica
- validações simples (relatório)
- correção do preço oficial via tabela `produtos` **preservando o preço original**
- persistência no SQLite


In [1]:
import pandas as pd
import sqlite3

In [2]:
arquivo_excel = "../dados_raw/base_teste_fato_original.xlsx"

df_vendas= pd.read_excel(arquivo_excel, sheet_name="Vendas")
df_produtos= pd.read_excel(arquivo_excel, sheet_name="Produtos")
df_clientes= pd.read_excel(arquivo_excel, sheet_name="Clientes")
df_metas= pd.read_excel(arquivo_excel, sheet_name="Metas")

display(df_vendas.head())


,Data,ID_Produto,ID_Cliente,Canal_Venda,Quantidade,Valor_Unitario,Valor_Total
0,2024-06-22,3,27,Marketplace,4,99.9,399.6
1,2024-05-19,4,43,WhatsApp,3,79.9,239.7
2,2024-03-16,7,12,WhatsApp,1,79.9,79.9
3,2024-02-28,10,20,WhatsApp,5,249.9,1249.5
4,2024-04-05,4,48,WhatsApp,2,199.9,399.8


In [3]:
# 2) Padronização de colunas
def snake_case(s: str) -> str:
    s = s.strip().lower()
    s = s.replace(" ", "_").replace("-", "_")
    return s

df_vendas.columns = [snake_case(c) for c in df_vendas.columns]
df_produtos.columns = [snake_case(c) for c in df_produtos.columns]
df_clientes.columns = [snake_case(c) for c in df_clientes.columns]
df_metas.columns = [snake_case(c) for c in df_metas.columns]

# Renomes recomendados pra consistência
df_vendas = df_vendas.rename(columns={"data": "data_venda"})
df_metas = df_metas.rename(columns={"mes": "ano_mes"})

print("Colunas vendas:", df_vendas.columns.tolist())
print("Colunas produtos:", df_produtos.columns.tolist())
print("Colunas metas:", df_metas.columns.tolist())

Colunas vendas: ['data_venda', 'id_produto', 'id_cliente', 'canal_venda', 'quantidade', 'valor_unitario', 'valor_total']
Colunas produtos: ['id_produto', 'nome_produto', 'categoria', 'custo_unitario', 'preco_venda']
Colunas metas: ['ano_mes', 'meta_faturamento']


In [4]:
# 3) Tipagem básica

# Datas
if "data_venda" in df_vendas.columns:
    df_vendas["data_venda"] = pd.to_datetime(df_vendas["data_venda"], errors="coerce")

# Numéricos em Vendas
for col in ["quantidade", "valor_unitario", "valor_total"]:
    if col in df_vendas.columns:
        df_vendas[col] = pd.to_numeric(df_vendas[col], errors="coerce")

# Numéricos em Produtos
for col in ["preco_venda", "custo_unitario"]:
    if col in df_produtos.columns:
        df_produtos[col] = pd.to_numeric(df_produtos[col], errors="coerce")

# Meta
if "meta_faturamento" in df_metas.columns:
    df_metas["meta_faturamento"] = pd.to_numeric(df_metas["meta_faturamento"], errors="coerce")

display(df_vendas.dtypes)

data_venda        datetime64[ns]
id_produto                 int64
id_cliente                 int64
canal_venda               object
quantidade                 int64
valor_unitario           float64
valor_total              float64
dtype: object

## Validações simples (relatório)
É um **checklist mínimo** para garantir que o dado é utilizável e para documentar problemas óbvios.

In [5]:
# 4) Validações mínimas (não altera dado, só reporta)

def relatorio_validacao(df, nome, chaves=None):
    out = {}
    out["tabela"] = nome
    out["linhas"] = len(df)
    out["colunas"] = df.shape[1]
    out["nulos_total"] = int(df.isna().sum().sum())
    out["nulos_por_coluna"] = df.isna().sum().to_dict()
    if chaves:
        out["duplicatas_chave"] = int(df.duplicated(subset=chaves).sum())
    return out

rel_vendas = relatorio_validacao(df_vendas, "vendas", chaves=None)  # pode ter múltiplas vendas por mesmo ID
rel_produtos = relatorio_validacao(df_produtos, "produtos", chaves=["id_produto"] if "id_produto" in df_produtos.columns else None)
rel_clientes = relatorio_validacao(df_clientes, "clientes", chaves=["id_cliente"] if "id_cliente" in df_clientes.columns else None)
rel_metas = relatorio_validacao(df_metas, "metas", chaves=["ano_mes"] if "ano_mes" in df_metas.columns else None)

relatorios = pd.DataFrame([rel_vendas, rel_produtos, rel_clientes, rel_metas])
display(relatorios[["tabela","linhas","colunas","nulos_total","duplicatas_chave"]])

# Checagens pontuais em Vendas (se as colunas existirem)
checks = {}

if "quantidade" in df_vendas.columns:
    checks["quantidade_<=0"] = int((df_vendas["quantidade"] <= 0).sum(skipna=True))

if "data_venda" in df_vendas.columns:
    checks["data_invalida"] = int(df_vendas["data_venda"].isna().sum())

if "id_produto" in df_vendas.columns and "id_produto" in df_produtos.columns:
    checks["id_produto_inexistente"] = int((~df_vendas["id_produto"].isin(df_produtos["id_produto"])).sum())

if "id_cliente" in df_vendas.columns and "id_cliente" in df_clientes.columns:
    checks["id_cliente_inexistente"] = int((~df_vendas["id_cliente"].isin(df_clientes["id_cliente"])).sum())

pd.DataFrame([checks])


,tabela,linhas,colunas,nulos_total,duplicatas_chave
0,vendas,200,7,0,NaN
1,produtos,10,5,0,0.0
2,clientes,50,5,0,0.0
3,metas,6,2,0,0.0


,quantidade_<=0,data_invalida,id_produto_inexistente,id_cliente_inexistente
0,0,0,0,0


## Correção mínima do preço (ponto crítico do desafio)

- **Preservar** o preço original registrado em `vendas.valor_unitario` (se existir)
- Definir o preço oficial como `produtos.preco_venda`
- **Não** calcular lucro/margem aqui (fica para o Notebook 02)

Resultado: `vendas_certificadas` enxuta e confiável.


In [6]:
# 5) Certificação de preço

# Base mínima esperada em vendas
cols_necessarias = ["data_venda", "id_produto", "id_cliente", "canal_venda", "quantidade"]
faltando_cols = [c for c in cols_necessarias if c not in df_vendas.columns]
if faltando_cols:
    raise ValueError(f"Colunas necessárias ausentes em Vendas: {faltando_cols}")

# Preserva o original, se existir
if "valor_unitario" in df_vendas.columns:
    df_vendas["preco_original_registrado"] = df_vendas["valor_unitario"]
else:
    df_vendas["preco_original_registrado"] = pd.NA

# Junta preço oficial (tabela produtos)
if "id_produto" not in df_produtos.columns or "preco_venda" not in df_produtos.columns:
    raise ValueError("Produtos precisa ter colunas: id_produto e preco_venda")

df_vendas = df_vendas.merge(
    df_produtos[["id_produto", "preco_venda"]],
    on="id_produto",
    how="left"
)

df_vendas = df_vendas.rename(columns={"preco_venda": "preco_unitario_oficial"})

# Mantém só as colunas essenciais da tabela certificada
vendas_certificadas = df_vendas[
    ["data_venda", "id_produto", "id_cliente", "canal_venda", "quantidade",
     "preco_original_registrado", "preco_unitario_oficial"]
].copy()

display(vendas_certificadas.head())


,data_venda,id_produto,id_cliente,canal_venda,quantidade,preco_original_registrado,preco_unitario_oficial
0,2024-06-22,3,27,Marketplace,4,99.9,3999.9
1,2024-05-19,4,43,WhatsApp,3,79.9,149.9
2,2024-03-16,7,12,WhatsApp,1,79.9,699.9
3,2024-02-28,10,20,WhatsApp,5,249.9,349.9
4,2024-04-05,4,48,WhatsApp,2,199.9,149.9


In [7]:
# 6) Persistência no SQLite 

DB = "../database/dados_tratados.db"
with sqlite3.connect(DB) as conn:
    vendas_certificadas.to_sql("vendas_certificadas", conn, if_exists="replace", index=False)
    df_produtos.to_sql("produtos_certificados", conn, if_exists="replace", index=False)
    df_clientes.to_sql("clientes_certificados", conn, if_exists="replace", index=False)
    df_metas.to_sql("metas_certificadas", conn, if_exists="replace", index=False)

conn.close()
print("✅ Tabelas salvas no SQLite:")

✅ Tabelas salvas no SQLite:


In [8]:
with sqlite3.connect(DB) as conn:
    print(pd.read_sql("SELECT COUNT(*) AS linhas FROM vendas_certificadas", conn))
    # se tiver data
    # print(pd.read_sql("SELECT MAX(data_venda) AS max_data FROM vendas_certificadas", conn))

   linhas
0     200
